In [15]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
from urllib.parse import urljoin
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm

Scrape all forum links concurrently from search pages (184 pages)

In [4]:
def get_forum_links(search_page_url, base_url="https://www.agnoshealth.com"):
    try:
        res = requests.get(search_page_url, timeout=30)
        res.raise_for_status()
        soup = BeautifulSoup(res.text, "html.parser")
        links = [urljoin(base_url, a['href']) for a in soup.find_all("a", href=True) if "/forums/" in a['href']]
        return list(set(links))
    except Exception as e:
        print(f"Error: {e} | URL: {search_page_url}")
        return []

urls = [f"https://www.agnoshealth.com/forums/search?page={i}" for i in range(1, 185)]
all_post_links = []

# ใช้ ThreadPoolExecutor เพื่อ run พร้อมกันหลาย thread
max_workers = 3

with ThreadPoolExecutor(max_workers=max_workers) as executor:
    # ส่งงานทั้งหมด
    future_to_url = {executor.submit(get_forum_links, url): url for url in urls}
    
    # ใช้ tqdm แสดง progress
    for future in tqdm(as_completed(future_to_url), total=len(future_to_url), desc="Scraping forum links"):
        links = future.result()
        all_post_links.extend(links)

# กำจัดซ้ำ
all_post_links = list(set([url for url in all_post_links if url != 'https://www.agnoshealth.com/forums/search']))

print(f"Total forum links: {len(all_post_links)}")


Scraping forum links: 100%|██████████| 184/184 [09:41<00:00,  3.16s/it]

Total forum links: 1471


In [5]:
# save all post links into txt file

all_post_links = list(set([url for url in all_post_links if url != 'https://www.agnoshealth.com/forums/search']))

# บันทึกลงไฟล์ txt
with open("forum_links.txt", "w", encoding="utf-8") as f:
    for link in all_post_links:
        f.write(link + "\n")

print(f"Saved {len(all_post_links)} links to forum_links.txt")

Saved 1471 links to forum_links.txt


In [7]:
def scrape_forum(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, "html.parser")

    # ดึงกระทู้หลัก
    article = soup.find("article")

    title_tag = article.find("p", class_="font-bold")
    title = title_tag.text.strip() if title_tag else ""

    info_tag = article.find("p", class_="text-sm text-gray-500")
    info = info_tag.text.strip() if info_tag else ""

    tags = [li.text.strip() for li in article.find_all("li")]

    desc_tag = article.find("p", class_="text-sm text-gray-500 line-clamp-3")
    description = desc_tag.text.strip() if desc_tag else ""

    # status / likes
    likes_tag = article.find_all("p", class_="text-sm text-primary_blue-500")
    likes = likes_tag[0].text.strip() if likes_tag else "0"

    status_tag = article.find("span", class_="text-sm text-primary_blue-500")
    status = status_tag.text.strip() if status_tag else ""

    # ดึงคำตอบแพทย์ (ถ้ามี)
    expert_answer, expert_name, expert_role = "", "", ""
    for li_tag in soup.find_all("li"):
        if "shadow-blue_glow_small" not in li_tag.get("class", []):
            continue
        role_tag = li_tag.find("p", string=lambda t: "คำตอบโดยแพทย์" in t if t else False)
        if role_tag:
            expert_role = role_tag.text.strip()
            name_tag = li_tag.find("p", class_="font-bold")
            expert_name = name_tag.text.strip() if name_tag else ""
            content_tag = li_tag.find("p", class_="mt-4")

            expert_answer = content_tag.text.strip() if content_tag else ""
            break  # เลือกคำตอบแรกของแพทย์

    return {
        "link": url,
        "title": title,
        "info": info,
        "tags": tags,
        "description": description,
        "likes": likes,
        "status": status,
        "expert_role": expert_role,
        "expert_name": expert_name,
        "expert_answer": expert_answer
    }

scrape data from link forum

In [13]:
# Read links from txt file
with open("forum_links.txt", "r", encoding="utf-8") as f:
    all_post_links = [line.strip() for line in f if line.strip()]

print(f"Loaded {len(all_post_links)} links from forum_links.txt")

all_data = []
max_workers = 5  # ปรับตามจำนวน thread ที่ต้องการ

with ThreadPoolExecutor(max_workers=max_workers) as executor:
    future_to_url = {executor.submit(scrape_forum, url): url for url in all_post_links}
    
    for future in tqdm(as_completed(future_to_url), total=len(future_to_url), desc="Scraping forums"):
        url = future_to_url[future]
        try:
            data = future.result()
            all_data.append(data)
        except Exception as e:
            print(f"Error scraping {url}: {e}")

print(f"Scraped data from {len(all_data)} forums")


Loaded 1471 links from forum_links.txt


Scraping forums: 100%|██████████| 1471/1471 [06:37<00:00,  3.70it/s]

Scraped data from 1471 forums


In [16]:
df = pd.DataFrame(all_data)
df.to_excel("data.xlsx")

In [17]:
df.head()

,link,title,info,tags,description,likes,status,expert_role,expert_name,expert_answer
0,https://www.agnoshealth.com/forums/%E0%B8%9C%E...,ผื่นตามร่างกาย (Dermatitis unspecified),หญิง | อายุ 19 ปี,[ผื่นตามร่างกาย],อยากรู้ว่าเป็นโรคอะไรและวิธีการรักษาค่ะ,0,,,,
1,https://www.agnoshealth.com/forums/%E0%B8%A3%E...,ริดสีดวงทวาร (Hemorrhoids),ชาย | อายุ 21 ปี,"[ถ่ายเป็นเลือดสด, ถ่ายปนเลือด, ริดสีดวงทวาร]",คือเป็นกังวลมากครับช่วงที่ผ่านเคยมีอาการถ่ายเป...,0,แพทย์ตอบคำปรึกษาแล้ว,คำตอบโดยแพทย์ผู้เชี่ยวชาญ,นราวิชญ์ จันทวรรณ,อายุ 21 ปี โอกาสเป็นมะเร็งน้อยอยู่แล้วครับ อาก...
2,https://www.agnoshealth.com/forums/%E0%B8%AD%E...,อ่อนเพลียทั่วไป (Fatigue unspecified),ชาย | อายุ 31 ปี,"[อ่อนเพลียทั่วไป, ง่วงตลอดเวลา, หลับมากกว่าปกติ]",มีอาการทำพฤติกรรมซ้ำๆจนติดเป็นนิสัย ตอนนี้ทำให...,0,,,,
3,https://www.agnoshealth.com/forums/%E0%B9%80%E...,เชื้อราในช่องคลอด (Candida vaginitis),หญิง | อายุ 17 ปี,"[ตกขาวผิดปกติ, เชื้อราในช่องคลอด]",อยากรู้ว่าตรวจโรคนี้แบบไม่ตรวจภายในได้มั้ยคะ,0,,,,
4,https://www.agnoshealth.com/forums/%E0%B8%81%E...,กระเพาะปัสสาวะอักเสบ (Cystitis),หญิง | อายุ 20 ปี,"[ปัสสาวะออกเยอะ, ปัสสาวะบ่อย, กระเพาะปัสสาวะอั...",คุณหมอคะ ช่วงนี้ทานยาดุเดือดมากค่ะ ทั้งยาต้านเ...,0,แพทย์ตอบคำปรึกษาแล้ว,คำตอบโดยแพทย์ผู้เชี่ยวชาญ,นราวิชญ์ จันทวรรณ,ค่าGFR ที่เริ่มจะมีปัญหาคือ น้อยกว่า 45 ครับ
